<a href="https://colab.research.google.com/github/aebongbing/ESAA/blob/main/ESAA_9%EC%9B%94_15%EC%9D%BC_%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# lamda 식을 이용할때 여러 개의 값을 인자로 사용해야 할 경우에는 보통 map() 함수를 이용한다
import numpy as np
import pandas as pd
a=[1,2,3]
squares=map(lambda x : x**2, a)
list(squares)

[1, 4, 9]

In [3]:
#apply()를 lambda식에 적용해보기
tdf=pd.read_csv('/content/train.csv')
tdf['Name_len']=tdf['Name'].apply(lambda x:len(x))
tdf[['Name','Name_len']].head(3)

,Name,Name_len
0,"Braund, Mr. Owen Harris",23
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",51
2,"Heikkinen, Miss. Laina",22


In [4]:
#if else 절을 이용한 복잡한 가공
tdf['Child_Adult']=tdf['Age'].apply(lambda x :'Child' if x <= 15  else 'Adult') # if 식은 반환값 먼저 else 식은 반환값이 나중에
tdf[['Age','Child_Adult']].head(8)

,Age,Child_Adult
0,22.0,Adult
1,38.0,Adult
2,26.0,Adult
3,35.0,Adult
4,35.0,Adult
5,NaN,Adult
6,54.0,Adult
7,2.0,Child


In [5]:
#elif는 지원하지 않음 이를 사용하기 위해서는 ()로 else절을 내포한 다음 ()안에서 다시 else if 적용해야함
tdf['Age_cat']=tdf['Age'].apply(lambda x : 'Child' if x<=15 else ('Adult' if x<= 60 else 'Elderly'))
tdf['Age_cat'].value_counts

<bound method IndexOpsMixin.value_counts of 0        Adult
1        Adult
2        Adult
3        Adult
4        Adult
        ...   
886      Adult
887      Adult
888    Elderly
889      Adult
890      Adult
Name: Age_cat, Length: 891, dtype: object>

In [6]:
#elif를 많이 써야 하거나 switch case문의 경우, 별도의 함수를 만드는 것이 편함
#나이에 따라 세분화된 분류를 수행하는 함수 생성

def get_category(age):
  cat=''
  if age <= 5: cat='Baby'
  elif age <=12:cat='Child'
  elif age <=18:cat='Teenager'
  elif age <=25:cat='Student'
  elif age <=35:cat='Young Adult'
  elif age <=60:cat='Adult'
  else: cat = 'Elderly'
  return cat
#lambda 식으로 위에서 생성한 get_category()함수를 반환값으로 지정
#get_category(x)는 입력값으로 'Age' 칼럼 값을 받아서 해당하는 cat 반환
tdf['Age_cat']=tdf['Age'].apply(lambda x : get_category(x))
tdf[['Age','Age_cat']].head()

,Age,Age_cat
0,22.0,Student
1,38.0,Adult
2,26.0,Young Adult
3,35.0,Young Adult
4,35.0,Young Adult


실제로 ML 모델을 생성하고 예측을 수행하는데에 있어서 ML알고리즘이 차지하는 비중고바 데이터를 전처리하고 적절한 피쳐를 가공하고 추출하는 부분이 훨씬 많은 비중을 차지합니다




<strong> * 사이킷런의 소개와 특징 </strong>

* 사이킷런(scikit-learn)
- 가장 많이 사용되는 파이썬 머신러닝 라이브러리
- 가장 파이썬스러운 API를 제공
- 머신러닝을 위한 다양한 알고리즘 제공
- 개발을 위한 편리한 프레임워크와 API 제공
-오랜기간 다양한 환경의 실전에서 사용된 성숙한 라이브러리


In [7]:
#사이킷런 설치하기
import sklearn as sk
#버전 확인하기
print(sk.__version__)

1.2.2


* 붓꽃 품종 예측하기: 붓꽃의 데이터 세트로 붓꽃의 품종을 ***분류(Classification)***
 - 분류(Classification)는 대표적인 지도학습(Supervised Learning) 중 하나
 - 지도학습은 명확한 답이 주어진 데이터를 먼저 학습한 뒤 미지의 정답을 예측함
 - 학습을 위해 주어진 데이터 세트 : 학습데이터
 - 머신러닝 모델의 예측 성능을 평가하기 위해 별도로 주어진 데이터: 데이터 세트
 - sklearn.datasets: 사이킷런이 자체 제공하는 데이터 세트를 생성하는 모듈의 모임
 - sklearn.tree: 트리 기반 ML 알고리즘을 구현한 클래스의 모임
 - sklearn.model_selection: 학습 데이터와 검증 데이터, 예측 데이터로 데이터를 분리하거나 최적의 하이퍼 파라미터로 평가하기 위한 다양한 모듈의 모임
 - 하이퍼 파라미터: 머신러닝 알고리즘별로 최적의 학습을 위해 직접 입력하는 파라미터로 이것을 통해 머신러닝 알고리즘의 성능을 튜닝가능

In [8]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [9]:
import pandas as pd

#붓꽃 데이터 세트 로딩
iris=load_iris()

#iris.data는 Iris 데이터 세트에서 피쳐만으로 된 데이터를 numpy로 가짐
iris_data=iris.data

#iris_type은 붓꽃 데이터 세트에서 레이블 (결정값) 데이터를 numpy로 가짐
iris_label=iris.target
print('iris target값:', iris_label)
print('iris target명:', iris.target_names)

#붓꽃 데이터 세트를 자세히 보기 위해 데이터프레임 변환
iris_df=pd.DataFrame(data=iris_data, columns=iris.feature_names)
iris_df['label']=iris.target
iris_df.head(3)

iris target값: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
iris target명: ['setosa' 'versicolor' 'virginica']


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0


학습용 데이터와 테스트용 데이터는 반드시 분리해야 함 왜냐하면 학습 데이터로 학습된 모델의 성능을 편가하려면 테스트 데이터 세트가 필요하기 때문에
이를 위해 사이킷런은 train_test_split()을 제공함

train_test_split()은 학습 데이터와 테스트 데이터를 test_size 파라미터 입력값의 비율로 분할함

In [10]:
X_train, X_test, y_train, y_test=train_test_split(iris_data, iris_label, test_size=0.2, random_state=11)

* 학습데이터를 확보했으니 해당 데이터를 기반으로 머신러닝 분류 알고리즘의 하나인 <strong>의사 결정 트리</strong>를 이용하여 학습과 예측을 수행

In [11]:
#DecisionTreeClassifier 객체 생성
dt_clf=DecisionTreeClassifier(random_state=11)
#학습 수행
dt_clf.fit(X_train, y_train)

DecisionTreeClassifier(random_state=11)

In [13]:
#학습이 완료된 DecisionTreeClassifier 객체에서 테스트 데이터 세트로 예측 수행
pred=dt_clf.predict(X_test)

In [15]:
#예측 결과를 기반으로 의사 결정 트리 기반의 DecsionTreeClassifier의 에측 성능을 평가하기
#정확도 측정=예측 결과가 실제 레이블 값과 얼마나저왁하게 맞는지를 평가하는 지표
from sklearn.metrics import accuracy_score
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))

예측 정확도: 0.9333


* 예측 프로세스 정리
 - 1. 데이터 세트 분리: 데이터를 학습 데이터와 테스트 데이터로 분리
 - 2. 모델 학습: 학습 데이터를 기반으로 ML 알고리즘 적용시켜 모델 학습시킴
 - 3. 예측 수행: 학습된 ML모델을 이용하여 테스트 데이터의 분류 예측
 - 4. 평가: 예측된 결과값과 테스트 데이터의 실제 결과값을 비교 ML모델 성능평가

* 사이킷런 기반의 프레임워크 익히기
 - Estimator: 지도학습의 모든 알고리즘 구현한 클래스, fit(), predict() 내부 구현, 분류와 회귀 두 축으로 이루어짐
 - 비지도 학습(ex. 차원 축소,클러스터링, 피처추출)등을 구현한 클래스: fit(), transform(), fit_transform()

 * 내장된 예제 데이터 세트
  - data: 피처의 대이터 세트
  - target: 분류 - 레이블 값, 회귀 - 숫자 결과값 데이터 세트
  - target_names: 개별 레이블 이름
  - feature_names: 피처의 이름
  - DESCR: 데이터 세트에 대한 설명 및 각 피처의 설명

In [19]:
# 붓꽃 데이터 세트 생성
from sklearn.datasets import load_iris
iris_data= load_iris()
print(type(iris_data))

#데이터 세트의 key 값 확인
keys=iris_data.keys()
print('붓꽃 데이터 세트의 키들:',keys)

#키들이 가리키는 값 출력
print('\n feature_names의 type:', type(iris_data.feature_names))
print('feature_names의 shape:', len(iris_data.feature_names))
print(iris_data.feature_names)

print('\n target_names의 type:', type(iris_data.target_names))
print('target_names의 shape:', len(iris_data.target_names))
print(iris_data.target_names)

print('\n data의 type:', type(iris_data.data))
print('data 의 shape:', iris_data.data.shape)
print(iris_data['data'])

print('\n target의 type:', type(iris_data.target))
print('target의 shape:', iris_data.target.shape)
print(iris_data.target)

<class 'sklearn.utils._bunch.Bunch'>
붓꽃 데이터 세트의 키들: dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

 feature_names의 type: <class 'list'>
feature_names의 shape: 4
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']

 target_names의 type: <class 'numpy.ndarray'>
target_names의 shape: 3
['setosa' 'versicolor' 'virginica']

 data의 type: <class 'numpy.ndarray'>
data 의 shape: (150, 4)
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2]
 [5.1 3.3 1.7 0.5]
 [4.8 3.4 1.9 0.2]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.2 3.5 1.5 0.

* Model Selection 모듈 소개
 - 학습데이터와 테스트 데이터 세트를 분리
 - 교차 검증 분할 및 평가
 - Estimator 의 하이퍼 파라미터를 튜닝하기 위한 함수&클래스 제공

In [21]:
#데이터를 분리하지 않은 경우
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

iris=load_iris()
dt_clf=DecisionTreeClassifier()
train_data=iris.data
train_label=iris.target
dt_clf.fit(train_data, train_label)

#학습 데이터 세트로 예측 수행
pred=dt_clf.predict(train_data)
print('예측 정확도:', accuracy_score(train_label,pred))

예측 정확도: 1.0


* 학습/테스트 데이터 분리-train_test_split()
 - test_size: 테스트 데이터 세트의 크기 디폴트=0.25
 - train_size: 학습용 데이터 세트의 크기
 - shuffle: 데이터 섞을지 말지 결정 디폴트=true
 - random_state: 난수값 생성
 - train_test_split: 튜플 형태의 반환값

In [23]:
#학습/테스트 데이터 세트 분리하기
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

dt_clf=DecisionTreeClassifier()
iris=load_iris()

X_train, X_test, y_train, y_test=train_test_split(iris_data.data, iris_data.target, test_size=0.3, random_state=121)

#예측 정확도 측정하기
dt_clf.fit(X_train,y_train)
pred=dt_clf.predict(X_test)
print('예측 정확도: {0: .4f}'. format(accuracy_score(y_test, pred)))

예측 정확도:  0.9556


* 교차 검증
 - 데이터를 분리해도 과적합에 취약한 약점이 있음
 - 과적합: 모델이 학습 데이터에만 과도하게 최적화되어 실제 예측을 다른 테스트 데이터로 수행할 때 예측 성능이 과도하게 떨어지는 것
 - 교차검증: 이러한 데이터 편중을 막기 위해 별도의 여러 세트로 구성된 학습 데이터 세트와 검증 데이터 세트에서 학습과 평가를 수행하는 것

* k 폴드 교차 검증
  - 가장 보편적으로 사용되는 교차 검증 방법
  - k개의 데이터 세트를 만들어서 k번만큼 각 폴드 세트에 학습과 검증 평가를 반복적으로 수행함

In [25]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

iris=load_iris()
features=iris.data
label=iris.target
dt_clf=DecisionTreeClassifier(random_state=156)

#5개의 폴드 세트로 분리하는 kFold 객체와 폴드 세트별 정확도를 담을 리스트 객체 생성
kfold=KFold(n_splits=5)
cv_accuracy=[]
print('붓꽃 데이터 세트 크기',features.shape[0])

붓꽃 데이터 세트 크기 150


In [26]:
#교차 검증 수행시 마다 학습과 검증을 반복해 예측 정확도를 측정

n_iter=0

#KFold 객체의 split()을 호출하면 폴드별 학습용, 검증용 테스트의 로우 인덱스를 array로 변환
for train_index, test_index, in kfold.split(features):
  #kfold.split으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
  X_train, X_test=features[train_index], features[test_index]
  y_train, y_test=label[train_index], label[test_index]
  #학습 및 예측
  dt_clf.fit(X_train, y_train)
  pred=dt_clf.predict(X_test)
  n_iter+=1
  #반복 시마다 정확도 측정
  accuracy=np.round(accuracy_score(y_test,pred),4)
  train_size=X_train.shape[0]
  test_size=X_test.shape[0]
  print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'.format(n_iter, accuracy, train_size, test_size))
  print('#{0} 검증 세트 인덱스: {1}'.format(n_iter,test_index))
  cv_accuracy.append(accuracy)

#개별 iteration별 정확도를 합하여 평균 정확도 계산
print('\n## 평균 검증 정확도:', np.mean(cv_accuracy))


#1 교차 검증 정확도 :1.0, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#1 검증 세트 인덱스: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

#2 교차 검증 정확도 :0.9667, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#2 검증 세트 인덱스: [30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

#3 교차 검증 정확도 :0.8667, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#3 검증 세트 인덱스: [60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

#4 교차 검증 정확도 :0.9333, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#4 검증 세트 인덱스: [ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

#5 교차 검증 정확도 :0.7333, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#5 검증 세트 인덱스: [120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

## 평균 검증 정확도: 0.9


In [27]:
#stratified K 폴드: 불균형한 분포도를 가진 레이블 데이터 집합에 쓰임
#K 폴드의 문제점 확인
import pandas as pd
iris=load_iris()
iris_df=pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['label']=iris.target
iris_df['label'].value_counts()

0    50
1    50
2    50
Name: label, dtype: int64

In [30]:
kfold=KFold(n_splits=3)
n_iter=0
for train_index, test_index in kfold.split(iris_df):
  n_iter+=1
  label_train=iris_df['label'].iloc[train_index]
  label_test=iris_df['label'].iloc[test_index]
  print('## 교차 검증: {0}'.format(n_iter))
  print('학습 레이블 데이터 분포:\n', label_train.value_counts())
  print('검증 레이블 데이터 분포:\n', label_test.value_counts())

## 교차 검증: 1
학습 레이블 데이터 분포:
 1    50
2    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    50
Name: label, dtype: int64
## 교차 검증: 2
학습 레이블 데이터 분포:
 0    50
2    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    50
Name: label, dtype: int64
## 교차 검증: 3
학습 레이블 데이터 분포:
 0    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    50
Name: label, dtype: int64


학습 레이블은 1,2 밖에 없으므로 0의 경우는 전혀 학습하지 못함
이런 유형으로 교차 검증 데이터 세트를 분할하면 검증 예측 정확도는 0이 됨

In [31]:
from sklearn.model_selection import StratifiedKFold
skf=StratifiedKFold(n_splits=3)
n_iter=0
for train_index, test_index in skf.split(iris_df, iris_df['label']):
  n_iter+=1
  label_train=iris_df['label'].iloc[train_index]
  label_test=iris_df['label'].iloc[test_index]
  print('## 교차 검증: {0}'.format(n_iter))
  print('학습 레이블 데이터 분포:\n', label_train.value_counts())
  print('검증 레이블 데이터 분포:\n', label_test.value_counts())

## 교차 검증: 1
학습 레이블 데이터 분포:
 2    34
0    33
1    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    17
1    17
2    16
Name: label, dtype: int64
## 교차 검증: 2
학습 레이블 데이터 분포:
 1    34
0    33
2    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    17
2    17
1    16
Name: label, dtype: int64
## 교차 검증: 3
학습 레이블 데이터 분포:
 0    34
1    33
2    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    17
2    17
0    16
Name: label, dtype: int64


In [32]:
dt_clf=DecisionTreeClassifier(random_state=156)

skfold=StratifiedKFold(n_splits=3)
n_iter=0
cv_accuracy=[]

#StratifiedKFold split() 호출시 반드시 레이블 데이터 세트 추가 입력
for train_index, test_index in skfold.split(features, label):
  #split()으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
  X_train, X_test=features[train_index],features[test_index]
  y_train, y_test=label[train_index], label[test_index]
  #학습 및 예측
  dt_clf.fit(X_train, y_train)
  pred=dt_clf.predict(X_test)

  #반복시마다 정확도 측정
  n_iter+=1
  accuracy=np.around(accuracy_score(y_test, pred),4)
  train_size=X_train.shape[0]
  test_size=X_test.shape[0]
  print('\n#{0} 교차 검증 정확도: {1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'.format(n_iter,accuracy, train_size, test_size))
  print('#{0} 검증 세트 인덱스: {1}'.format(n_iter, test_index))
  cv_accuracy.append(accuracy)

#교차 검증별 정확도 및 평균 정확도 계산
print('\n## 교차 검증별 정확도:', np.round(cv_accuracy,4))
print('## 평균 검증 정확도:', np.round(np.mean(cv_accuracy),4))


#1 교차 검증 정확도: 0.98, 학습 데이터 크기: 100, 검증 데이터 크기: 50
#1 검증 세트 인덱스: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115]

#2 교차 검증 정확도: 0.94, 학습 데이터 크기: 100, 검증 데이터 크기: 50
#2 검증 세트 인덱스: [ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132]

#3 교차 검증 정확도: 0.98, 학습 데이터 크기: 100, 검증 데이터 크기: 50
#3 검증 세트 인덱스: [ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]

## 교차 검증별 정확도: [0.98 0.94 0.98]
## 평균 검증 정확도: 0.9667


In [33]:
#교차 검증을 보다 간편하게-cross_val_score()
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.datasets import load_iris

iris_data=load_iris()
dt_clf=DecisionTreeClassifier(random_state=156)

data=iris_data.data
label=iris_data.target

#성능 지표는 정확도 (accuracy), 교차 검증 세트는 3개
scores=cross_val_score(dt_clf, data, label, scoring='accuracy', cv=3)
print('교차 검증별 정확도:', np.round(scores,4))
print('평균 검증별 정확도:', np.round(np.mean(scores), 4))

교차 검증별 정확도: [0.98 0.94 0.98]
평균 검증별 정확도: 0.9667


In [34]:
#GridSearchCV-교차 검증과 최적 하이퍼 파라미터 튜닝을 한번에
grid_parameters={'max_depth':[1,2,3],
                 'min_samples_split':[2,3]}

In [35]:
#예제
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

#데이터를 로딩하고 학습 데이터와 테스트 데이터를 분리
iris_data=load_iris()
X_train, X_test, y_train, y_test=train_test_split(iris_data.data, iris_data.target, test_size=0.2, random_state=121)
dtree=DecisionTreeClassifier()

##파라미터를 딕셔너리 형태로 설정
parameters={'max_depth':[1,2,3], 'min_samples_split':[2,3]}

In [36]:
import pandas as pd

#param_grid의 하이퍼 파라미터를 3개의 train, test set fold로 나누어 테스트 수행 설정
### refit=True 가 default임 True면 가장 좋은 파라미터 설정으로 재학습시킴
grid_dtree=GridSearchCV(dtree, param_grid=parameters, cv=3, refit=True)

#붓꽃 학습 데이터로 param_grid의 하이퍼 파라미터를 순차적으로 학습/평가
grid_dtree.fit(X_train, y_train)

#GridSearchCV 결과를 추출해 DataFrame으로 변환
scores_df=pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', 'split0_test_score', 'split1_test_score', 'split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


In [38]:
#최적의 하이퍼 파라미터 값과 그때의 정확도를 알아보겠다
print('GridSearchCV 최적 파라미터:', grid_dtree.best_params_)
print('GridSearchCV 최고 정확도:{0:.4f}'.format(grid_dtree.best_score_))

GridSearchCV 최적 파라미터: {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도:0.9750


In [40]:
#GridSearchCV의 refit으로 이미 학습된 estimator 반환
estimator=grid_dtree.best_estimator_

#GridSearchCV의 best_estimator_는 이미 최적 학습이 됐으므로 별도 학습이 필요없음
pred=estimator.predict(X_test)
print('테스트 데이터 세트 정확도:{0:.4f}'.format(accuracy_score(y_test, pred)))

테스트 데이터 세트 정확도:0.9667


In [41]:
#데이터 전처리
##데이터 인코딩-레이블 인코딩
from sklearn.preprocessing import LabelEncoder

items=['TV', '냉장고', '전자레인지', '컴퓨터', '선풍기', '선풍기', '믹서', '믹서']

#LabelEncoder를 객체로 생성한 후 fit()과 transform()으로 레이블 인코딩 수행
encoder=LabelEncoder()
encoder.fit(items)
labels=encoder.transform(items)
print('인코딩 변환값:', labels)

인코딩 변환값: [0 1 4 5 3 3 2 2]


In [42]:
print('인코딩 클래스:', encoder.classes_)

인코딩 클래스: ['TV' '냉장고' '믹서' '선풍기' '전자레인지' '컴퓨터']


In [43]:
print('디코딩 원본값:', encoder.inverse_transform([4,5,2,0,1,1,3,3]))

디코딩 원본값: ['전자레인지' '컴퓨터' '믹서' 'TV' '냉장고' '냉장고' '선풍기' '선풍기']


In [44]:
#원-핫 인코딩
from sklearn.preprocessing import OneHotEncoder
import numpy as np

items=['TV', '냉장고', '전자레인지', '컴퓨터', '선풍기', '선풍기', '믹서', '믹서']

#2차원 ndarray로 변환
items=np.array(items).reshape(-1,1)

#원-핫 인코딩 적용
oh_encoder=OneHotEncoder()
oh_encoder.fit(items)
oh_labels=oh_encoder.transform(items)

#OneHotEncoder로 반환한 결과는 희소행렬이므로 toarray()를 이용하여 밀집 행렬로 변환
print('원-핫 인코딩 데이터')
print(oh_labels.toarray())
print('원-핫 인코딩 데이터 차원')
print(oh_labels.shape)

원-핫 인코딩 데이터
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]
원-핫 인코딩 데이터 차원
(8, 6)
